In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pyodbc
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
import warnings
warnings.filterwarnings('ignore')

SQLconnection = pyodbc.connect( 'DRIVER={SQL Server};SERVER=172.16.1.38;DATABASE=dbAlgebra;UID=cscheihing;PWD=ZXCasdqwe123' )
#Datos para buscar información
fechaISO="20191120"
tipoCurva = 'clp' #clp
SiOficial = '1'



Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'SQL Server' : file not found (0) (SQLDriverConnect)")

In [ ]:
sqlGammas='''
SELECT Ancla,y0 as LargoPlazo,y1 as  ConcavMuyCortoPlazo,y2 as CortoPlazo,y3 as CortoPlazo2 ,y4 as ConvexMuyCortoPlazo FROM dbInvestigacion.dbo.TdCurvasRF_Parametros
where Fecha = '@fecha' and Tipo = '@TipoCurva' and SiOficial = '@SiOficial' Order by Inicial1_Final2 '''
sqlGammas=sqlGammas.replace("@fecha",fechaISO)
sqlGammas=sqlGammas.replace("@TipoCurva",tipoCurva)
sqlGammas=sqlGammas.replace("@SiOficial",SiOficial)
dataGammas = pd.io.sql.read_sql(sqlGammas, SQLconnection)


sqlInfo='''
SELECT Fecha,FechaCupon,Plazo,Nemotecnico,Flujo,TirBono,Base1,Base2,TirBonoOriginal,ReajusteProyectado FROM dbInvestigacion.dbo.TdCurvasRF_Inputs
where Fecha = '@fecha' and Tipo = '@TipoCurva' and SiOficial = '@SiOficial' Order by Nemotecnico, Plazo'''
sqlInfo=sqlInfo.replace("@fecha",fechaISO)
sqlInfo=sqlInfo.replace("@TipoCurva",tipoCurva)
sqlInfo=sqlInfo.replace("@SiOficial",SiOficial)
dataInfo = pd.io.sql.read_sql(sqlInfo, SQLconnection)



print(dataGammas.to_string())




In [ ]:
print("Ancla: El ancla mueve la parte izquierda de la curva, desde donde parte")
print("LargoPlazo: Mueve el largo plazo, haciendo que se mueva el corto plazo ligeramente")
print("ConcavMuyCortoPlazo: Hace la curva en el muy corto plazo mas concavo al disminuirlo, haciendo que el largo plazo se mueva ligeramente")
print("CortoPlazo: Sube el corto plazo al incrementarlo, destruyendo la concavidad o convexidad que exista en el muy corto plazo")
print("CortoPlazo2: Hace lo mismo que CortoPlazo")
print("ConvexMuyCortoPlazo: Hace la curva en el muy corto plazo mas convexo al disminuirlo, haciendo que el largo plazo se mueva ligeramente")

numerador = dataInfo.assign(col = dataInfo['Plazo']/365 * dataInfo['Flujo']).groupby(['Nemotecnico'],as_index=False).col.sum()
denominador = dataInfo.groupby(['Nemotecnico'])['Flujo'].sum()
TirBono = dataInfo.groupby(['Nemotecnico']).mean()["TirBono"]


x = np.linspace(0, 31.34, 1000)
maximo = max(dataInfo['Plazo'])

def widget_grafico(Ancla,LargoPlazo,ConcavMuyCortoP,CortoPlazo,CortoPlazo2,ConvexMuyCortoP):
    
    dataGammas.values[1,0] = Ancla
    dataGammas.values[1,1] = LargoPlazo
    dataGammas.values[1,2] = ConcavMuyCortoP
    dataGammas.values[1,3] = CortoPlazo
    dataGammas.values[1,4] = CortoPlazo2
    dataGammas.values[1,5] = ConvexMuyCortoP
        
    
    
    yHoy = dataGammas.values[1,1]+ (dataGammas.values[1,0]-dataGammas.values[1,1])*(1-np.exp(-x/dataGammas.values[1,2]))*(dataGammas.values[1,2]/x)+dataGammas.values[1,3]*((1-np.exp(-x/dataGammas.values[1,2]))*(dataGammas.values[1,2]/x)-np.exp(-x/dataGammas.values[1,2])) +dataGammas.values[1,4]*((1-np.exp(-x/dataGammas.values[1,5]))*(dataGammas.values[1,5]/x)-np.exp(-x/dataGammas.values[1,5]))
    yHoy[0] = dataGammas.values[1,0]
    yAyer = dataGammas.values[0,1]+ (dataGammas.values[0,0]-dataGammas.values[0,1])*(1-np.exp(-x/dataGammas.values[0,2]))*(dataGammas.values[0,2]/x)+dataGammas.values[0,3]*((1-np.exp(-x/dataGammas.values[0,2]))*(dataGammas.values[0,2]/x)-np.exp(-x/dataGammas.values[0,2])) +dataGammas.values[0,4]*((1-np.exp(-x/dataGammas.values[0,5]))*(dataGammas.values[0,5]/x)-np.exp(-x/dataGammas.values[0,5]))
    yAyer[0] = dataGammas.values[0,0]
    
    aux = 365*dataGammas.values[1,2]
    aux2 = 365*dataGammas.values[1,5]
    print("")

    
    plt.rcParams["figure.figsize"] = [16,9]
    plt.plot(numerador.values[:,1] / denominador, TirBono, 'go', label='Transas')
    plt.plot(x, yHoy, 'b', label='Hoy')
    plt.plot(x, yAyer, 'y', label='Ayer')
    plt.axis([0.0, 32.0, -3.0, 6.0])  
    plt.grid(True)
    plt.legend()
    plt.show()
    print(dataGammas.to_string())
    print('''
    UPDATE [dbAlgebra].[dbo].[TdCurvasSvensson] 
    SET ParametrosCalculados= \''''+str(dataGammas.values[1,0])+'''#'''+str(dataGammas.values[1,1])+'''#'''+str(aux)+'''#'''+str(dataGammas.values[1,3])+'''#'''+str(dataGammas.values[1,4])+'''#'''+str(aux2)+'''#'''+str(maximo)+'''\'
    where Fecha=''',fechaISO,'''and TipoCurva=''',tipoCurva)


    
slide1 = widgets.FloatSlider(min=dataGammas.values[1,0]-1,max=dataGammas.values[1,0]+1,step=0.1,value=dataGammas.values[1,0],continuous_update = False)
slide2 = widgets.FloatSlider(min=dataGammas.values[1,1]-1,max=dataGammas.values[1,1]+1,step=0.05,value=dataGammas.values[1,1],continuous_update = False)
slide3 = widgets.FloatSlider(min=dataGammas.values[1,2]-0.3,max=dataGammas.values[1,2]+0.3,step=0.001,value=dataGammas.values[1,2],continuous_update = False)
slide4 = widgets.FloatSlider(min=dataGammas.values[1,3]-3,max=dataGammas.values[1,3]+3,step=0.1,value=dataGammas.values[1,3],continuous_update = False)
slide5 = widgets.FloatSlider(min=dataGammas.values[1,4]-3,max=dataGammas.values[1,4]+3,step=0.1,value=dataGammas.values[1,4],continuous_update = False)
slide6 = widgets.FloatSlider(min=dataGammas.values[1,5]-0.3,max=dataGammas.values[1,5]+0.3,step=0.01,value=dataGammas.values[1,5],continuous_update = False)


#dataGammas.values[1,0]
interact(widget_grafico,Ancla=slide1,LargoPlazo=slide2,ConcavMuyCortoP=slide3,CortoPlazo=slide4,CortoPlazo2=slide5,ConvexMuyCortoP=slide6)
print("")

SQLconnection.close()